In [95]:
import pandas as pd
import os
import faiss

from sentence_transformers import SentenceTransformer

### data preparation

In [96]:
data_path = 'PATH'
os.chdir(data_path)
df_l = pd.read_csv('linkedin_sample.csv', nrows=100000)
df_r = pd.read_csv('burning_glass_sample.csv', nrows=100000)

# benchs = pd.read_csv('fuzzy_match_benchmark.csv', nrows=1000)

In [97]:
# decreasing the query dataset

### convert text to vectors
- we have other methods
- to be implemented...

In [98]:
def text2vec(df, col_name):
    text = df[col_name]
    encoder = SentenceTransformer('paraphrase-mpnet-base-v2')
    vectors = encoder.encode(text)
    return vectors

In [99]:
xb1 = text2vec(df_l, 'company_cleaned_std')
xq1 = text2vec(df_r, 'compname_std')

### search based on faiss

In [100]:
n, d = xb1.shape
nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [101]:
assert not index.is_trained
index.train(xb1)
assert index.is_trained

In [102]:
index.add(xb1)                  # add may be a bit slower as well
D, I = index.search(xq1, k)     # actual search            

In [ ]:
results = pd.DataFrame(I, columns=['index_1', 'index_2', 'index_3', 'index_4']) # this index number depends on parameter *k*

results['init_name'] = df_r['compname_std']

df_l['index_1'] = df_l.index
df_l['index_2'] = df_l.index
df_l['index_3'] = df_l.index
df_l['index_4'] = df_l.index

for i in range(1,5):
    results = pd.merge(results, df_l[[f'index_{i}', 'company_cleaned_std']], on=f'index_{i}', how='left')

# generate the fuzzy matching results
results.drop(['index_1', 'index_2', 'index_3', 'index_4'], inplace=True, axis=1)
results